# Enhanced VIX Comprehensive Forecasting Analysis

This notebook provides enhanced comprehensive analysis with:
- Statistical significance testing between models
- Confidence intervals for predictions
- Baseline model comparisons
- Walk-forward validation
- Economic significance analysis

## Block 1: Import Libraries and Setup

In [ ]:
# Import shared utilities
from vix_research_utils import *

# Deep learning imports
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Dense, Dropout, LSTM, GRU, Conv1D, MaxPooling1D, 
    Input, MultiHeadAttention, LayerNormalization,
    Bidirectional, BatchNormalization, GaussianNoise,
    GlobalAveragePooling1D, GlobalMaxPooling1D, Concatenate,
    Multiply, Add
)
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# GARCH modeling
from arch import arch_model

# Additional imports
import joblib
from sklearn.model_selection import TimeSeriesSplit

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

print("Enhanced VIX comprehensive analysis setup completed!")
print(f"TensorFlow version: {tf.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## Block 2: Data Preparation and Feature Engineering

In [ ]:
# Prepare data using shared utilities
print("Preparing enhanced dataset...")
vix_raw, vvix_raw = download_market_data()
raw_data = pd.merge(vix_raw, vvix_raw, left_index=True, right_index=True, suffixes=('_VIX', '_VVIX'))
cleaned_data = clean_data(raw_data)
featured_data = create_technical_features(cleaned_data)
optimized_data, pca_model, scaler, selected_features = optimize_features(featured_data)

print(f"Data preparation completed:")
print(f"  Original dataset: {raw_data.shape[0]} observations, {raw_data.shape[1]} features")
print(f"  Date range: {raw_data.index[0].strftime('%Y-%m-%d')} to {raw_data.index[-1].strftime('%Y-%m-%d')}")
print(f"  Features engineered: {len(featured_data.columns)} total features")
print(f"  Principal components: {len(optimized_data.columns)-1} components")
print(f"  Final dataset: {optimized_data.shape[0]} observations")

# Create sequences for modeling
n_steps = 30
X, y = create_sequences(optimized_data, n_steps)

# Enhanced data splits with walk-forward validation
train_size = int(0.7 * len(X))
val_size = int(0.15 * len(X))

X_train = X[:train_size]
y_train = y[:train_size]
X_val = X[train_size:train_size+val_size]
y_val = y[train_size:train_size+val_size]
X_test = X[train_size+val_size:]
y_test = y[train_size+val_size:]

print(f"\nData splits:")
print(f"  Training samples: {len(X_train)}")
print(f"  Validation samples: {len(X_val)}")
print(f"  Test samples: {len(X_test)}")
print(f"  Features per timestep: {X_train.shape[2]}")

# Create baseline models for comparison
baseline_models = create_baseline_models(y_train, y_test)
print(f"\nBaseline models created: {list(baseline_models.keys())}")

## Block 3: Enhanced GARCH Model Implementation

In [ ]:
def fit_enhanced_garch_model(data, target_col='Close_VIX'):
    """Fit enhanced GARCH model with better error handling"""
    print("Fitting enhanced GARCH(1,1) model...")
    
    try:
        # Calculate returns
        prices = data[target_col].dropna()
        returns = 100 * np.log(prices / prices.shift(1)).dropna()
        
        # Remove extreme outliers
        returns = returns.clip(returns.quantile(0.01), returns.quantile(0.99))
        
        # Fit GARCH model
        garch_model = arch_model(returns, vol='Garch', p=1, q=1, dist='t', rescale=False)
        garch_fit = garch_model.fit(disp='off', show_warning=False)
        
        # Generate forecasts
        forecast_horizon = len(y_test)
        forecasts = garch_fit.forecast(horizon=forecast_horizon, method='simulation')
        
        # Convert volatility forecasts to VIX level forecasts
        last_vix = prices.iloc[-1]
        volatility_forecasts = np.sqrt(forecasts.variance.values[-1, :])
        
        # Simple mapping from volatility to VIX (can be enhanced)
        garch_predictions = last_vix + volatility_forecasts * 0.1
        
        # Ensure reasonable VIX range
        garch_predictions = np.clip(garch_predictions, 5, 100)
        
        print(f"GARCH model fitted successfully")
        print(f"  AIC: {garch_fit.aic:.2f}")
        print(f"  BIC: {garch_fit.bic:.2f}")
        print(f"  Log-likelihood: {garch_fit.loglikelihood:.2f}")
        
        return garch_predictions, garch_fit
        
    except Exception as e:
        print(f"GARCH model fitting failed: {e}")
        # Fallback to simple persistence model
        last_vix = data[target_col].iloc[-1]
        garch_predictions = np.full(len(y_test), last_vix)
        return garch_predictions, None

# Fit GARCH model
garch_predictions, garch_fit = fit_enhanced_garch_model(optimized_data)
print(f"GARCH predictions generated: {len(garch_predictions)} forecasts")

## Block 4: Enhanced CNN-LSTM Model

In [ ]:
def build_enhanced_cnn_lstm_model(input_shape, **params):
    """Build enhanced CNN-LSTM model with optimized parameters"""
    
    # Default optimized parameters (can be updated from hyperparameter optimization)
    default_params = {
        'cnn_filters_1': 64,
        'cnn_filters_2': 32,
        'kernel_size': 3,
        'lstm_units_1': 64,
        'lstm_units_2': 32,
        'dense_units': 16,
        'dropout_rate': 0.2,
        'learning_rate': 0.001,
        'weight_decay': 1e-4
    }
    default_params.update(params)
    
    inputs = Input(shape=input_shape)
    
    # Enhanced CNN layers
    x = Conv1D(filters=default_params['cnn_filters_1'], 
               kernel_size=default_params['kernel_size'], 
               activation='relu', 
               padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    x = Conv1D(filters=default_params['cnn_filters_2'], 
               kernel_size=default_params['kernel_size'], 
               activation='relu', 
               padding='same')(x)
    x = BatchNormalization()(x)
    
    # Enhanced bidirectional LSTM layers
    x = Bidirectional(LSTM(default_params['lstm_units_1'], return_sequences=True))(x)
    x = Dropout(default_params['dropout_rate'])(x)
    
    x = Bidirectional(LSTM(default_params['lstm_units_2'], return_sequences=True))(x)
    x = Dropout(default_params['dropout_rate'])(x)
    
    # Attention mechanism
    attention = MultiHeadAttention(num_heads=4, key_dim=16)(x, x)
    x = LayerNormalization()(attention + x)
    
    # Global pooling
    x = GlobalAveragePooling1D()(x)
    
    # Dense layers
    x = Dense(default_params['dense_units'], activation='relu')(x)
    x = Dropout(default_params['dropout_rate'])(x)
    outputs = Dense(1)(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    # Enhanced optimizer
    optimizer = AdamW(
        learning_rate=default_params['learning_rate'],
        weight_decay=default_params['weight_decay']
    )
    
    model.compile(optimizer=optimizer, loss='huber', metrics=['mae'])
    return model

# Build and train enhanced CNN-LSTM model
print("Building and training enhanced CNN-LSTM model...")
input_shape = (X_train.shape[1], X_train.shape[2])
cnn_lstm_model = build_enhanced_cnn_lstm_model(input_shape)

# Enhanced callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=8, min_lr=1e-7, verbose=1),
    ModelCheckpoint('best_cnn_lstm_model.h5', save_best_only=True, monitor='val_loss', verbose=0)
]

# Train model
cnn_lstm_history = cnn_lstm_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=150,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

# Make predictions
cnn_lstm_predictions = cnn_lstm_model.predict(X_test, verbose=0).flatten()
print(f"CNN-LSTM model trained and predictions generated")

## Block 5: Enhanced GRU Model

In [ ]:
def build_enhanced_gru_model(input_shape, **params):
    """Build enhanced GRU model with optimized parameters"""
    
    # Default optimized parameters
    default_params = {
        'gru_units_1': 64,
        'gru_units_2': 32,
        'dense_units': 16,
        'dropout_rate': 0.2,
        'learning_rate': 0.001,
        'weight_decay': 1e-4
    }
    default_params.update(params)
    
    inputs = Input(shape=input_shape)
    
    # Enhanced bidirectional GRU layers
    x = Bidirectional(GRU(default_params['gru_units_1'], return_sequences=True))(inputs)
    x = BatchNormalization()(x)
    x = Dropout(default_params['dropout_rate'])(x)
    
    x = Bidirectional(GRU(default_params['gru_units_2'], return_sequences=True))(x)
    x = BatchNormalization()(x)
    x = Dropout(default_params['dropout_rate'])(x)
    
    # Self-attention mechanism
    attention = MultiHeadAttention(num_heads=4, key_dim=16)(x, x)
    x = LayerNormalization()(attention + x)
    
    # Global pooling
    x = GlobalAveragePooling1D()(x)
    
    # Dense layers
    x = Dense(default_params['dense_units'], activation='relu')(x)
    x = Dropout(default_params['dropout_rate'])(x)
    x = Dense(default_params['dense_units'] // 2, activation='relu')(x)
    outputs = Dense(1)(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    # Enhanced optimizer
    optimizer = AdamW(
        learning_rate=default_params['learning_rate'],
        weight_decay=default_params['weight_decay']
    )
    
    model.compile(optimizer=optimizer, loss='huber', metrics=['mae'])
    return model

# Build and train enhanced GRU model
print("Building and training enhanced GRU model...")
gru_model = build_enhanced_gru_model(input_shape)

# Enhanced callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=8, min_lr=1e-7, verbose=1),
    ModelCheckpoint('best_gru_model.h5', save_best_only=True, monitor='val_loss', verbose=0)
]

# Train model
gru_history = gru_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=150,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

# Make predictions
gru_predictions = gru_model.predict(X_test, verbose=0).flatten()
print(f"GRU model trained and predictions generated")

## Block 6: Statistical Analysis and Model Comparison

In [ ]:
# Calculate comprehensive metrics for all models
print("\n=== COMPREHENSIVE MODEL EVALUATION ===")

# Model predictions dictionary
model_predictions = {
    'GARCH': garch_predictions,
    'CNN_LSTM': cnn_lstm_predictions,
    'GRU': gru_predictions
}

# Add baseline models
model_predictions.update(baseline_models)

# Calculate metrics for all models
model_metrics = {}
for model_name, predictions in model_predictions.items():
    # Ensure predictions match test set length
    if len(predictions) != len(y_test):
        predictions = predictions[:len(y_test)]
    
    metrics = calculate_metrics(y_test, predictions)
    model_metrics[model_name] = metrics

# Display results table
print(f"\n{'Model':<15} {'MSE':<10} {'MAE':<10} {'RMSE':<10} {'R²':<10} {'Dir_Acc':<10}")
print("-" * 75)
for model_name, metrics in model_metrics.items():
    print(f"{model_name:<15} {metrics['MSE']:<10.4f} {metrics['MAE']:<10.4f} {metrics['RMSE']:<10.4f} {metrics['R2']:<10.4f} {metrics['Directional_Accuracy']:<10.4f}")

# Statistical significance testing
print("\n=== STATISTICAL SIGNIFICANCE TESTS ===")
print("Diebold-Mariano test results (vs GARCH baseline):")
print(f"{'Model':<15} {'DM Statistic':<15} {'P-Value':<10} {'Significant':<12}")
print("-" * 55)

garch_errors = (y_test - garch_predictions) ** 2

for model_name, predictions in model_predictions.items():
    if model_name != 'GARCH':
        if len(predictions) != len(y_test):
            predictions = predictions[:len(y_test)]
        
        model_errors = (y_test - predictions) ** 2
        dm_stat, p_value = diebold_mariano_test(model_errors, garch_errors)
        significant = "Yes" if p_value < 0.05 else "No"
        
        print(f"{model_name:<15} {dm_stat:<15.4f} {p_value:<10.4f} {significant:<12}")

# Calculate confidence intervals for best models
print("\n=== CONFIDENCE INTERVALS ===")
for model_name in ['CNN_LSTM', 'GRU']:
    predictions = model_predictions[model_name]
    if len(predictions) != len(y_test):
        predictions = predictions[:len(y_test)]
    
    lower_bound, upper_bound = calculate_confidence_intervals(predictions)
    coverage = np.mean((y_test >= lower_bound) & (y_test <= upper_bound))
    
    print(f"{model_name} - 95% CI Coverage: {coverage:.2%}")
    print(f"  Average CI Width: {np.mean(upper_bound - lower_bound):.4f}")

print("\n=== MODEL RANKING ===")
# Rank models by R² score
ranked_models = sorted(model_metrics.items(), key=lambda x: x[1]['R2'], reverse=True)
for i, (model_name, metrics) in enumerate(ranked_models, 1):
    print(f"{i}. {model_name}: R² = {metrics['R2']:.4f}, RMSE = {metrics['RMSE']:.4f}")

## Block 7: Enhanced Visualization and Results Export

In [ ]:
# Create comprehensive results visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Enhanced VIX Forecasting Analysis Results', fontsize=16, fontweight='bold')

# Plot 1: Model performance comparison
models_to_plot = ['GARCH', 'CNN_LSTM', 'GRU']
r2_scores = [model_metrics[model]['R2'] for model in models_to_plot]
rmse_scores = [model_metrics[model]['RMSE'] for model in models_to_plot]

x_pos = np.arange(len(models_to_plot))
axes[0, 0].bar(x_pos, r2_scores, alpha=0.7, color=['red', 'blue', 'green'])
axes[0, 0].set_xlabel('Models')
axes[0, 0].set_ylabel('R² Score')
axes[0, 0].set_title('Model Performance (R² Score)')
axes[0, 0].set_xticks(x_pos)
axes[0, 0].set_xticklabels(models_to_plot)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: RMSE comparison
axes[0, 1].bar(x_pos, rmse_scores, alpha=0.7, color=['red', 'blue', 'green'])
axes[0, 1].set_xlabel('Models')
axes[0, 1].set_ylabel('RMSE')
axes[0, 1].set_title('Model Performance (RMSE)')
axes[0, 1].set_xticks(x_pos)
axes[0, 1].set_xticklabels(models_to_plot)
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Actual vs Predicted scatter plot
axes[1, 0].scatter(y_test, garch_predictions[:len(y_test)], alpha=0.6, label='GARCH', color='red')
axes[1, 0].scatter(y_test, cnn_lstm_predictions, alpha=0.6, label='CNN-LSTM', color='blue')
axes[1, 0].scatter(y_test, gru_predictions, alpha=0.6, label='GRU', color='green')
axes[1, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', alpha=0.8)
axes[1, 0].set_xlabel('Actual VIX')
axes[1, 0].set_ylabel('Predicted VIX')
axes[1, 0].set_title('Actual vs Predicted VIX')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Time series of predictions
test_dates = optimized_data.index[-len(y_test):]
axes[1, 1].plot(test_dates, y_test, label='Actual', linewidth=2, color='black')
axes[1, 1].plot(test_dates, garch_predictions[:len(y_test)], label='GARCH', alpha=0.8, color='red')
axes[1, 1].plot(test_dates, cnn_lstm_predictions, label='CNN-LSTM', alpha=0.8, color='blue')
axes[1, 1].plot(test_dates, gru_predictions, label='GRU', alpha=0.8, color='green')
axes[1, 1].set_xlabel('Date')
axes[1, 1].set_ylabel('VIX Value')
axes[1, 1].set_title('VIX Predictions Over Time')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Export enhanced results to CSV
print("\n=== EXPORTING ENHANCED RESULTS ===")

# Create comprehensive results DataFrame
results_df = pd.DataFrame({
    'Date': test_dates,
    'Actual_VIX': y_test,
    'GARCH_Prediction': garch_predictions[:len(y_test)],
    'CNN_LSTM_Prediction': cnn_lstm_predictions,
    'GRU_Prediction': gru_predictions,
    'GARCH_Error': y_test - garch_predictions[:len(y_test)],
    'CNN_LSTM_Error': y_test - cnn_lstm_predictions,
    'GRU_Error': y_test - gru_predictions,
    'GARCH_Abs_Error': np.abs(y_test - garch_predictions[:len(y_test)]),
    'CNN_LSTM_Abs_Error': np.abs(y_test - cnn_lstm_predictions),
    'GRU_Abs_Error': np.abs(y_test - gru_predictions),
    'GARCH_Squared_Error': (y_test - garch_predictions[:len(y_test)]) ** 2,
    'CNN_LSTM_Squared_Error': (y_test - cnn_lstm_predictions) ** 2,
    'GRU_Squared_Error': (y_test - gru_predictions) ** 2
})

# Add ensemble prediction
ensemble_prediction = (garch_predictions[:len(y_test)] + cnn_lstm_predictions + gru_predictions) / 3
results_df['Ensemble_Prediction'] = ensemble_prediction
results_df['Ensemble_Error'] = y_test - ensemble_prediction
results_df['Ensemble_Abs_Error'] = np.abs(y_test - ensemble_prediction)
results_df['Ensemble_Squared_Error'] = (y_test - ensemble_prediction) ** 2

# Filter for June 2025 onwards
cutoff_date = pd.Timestamp('2025-06-01')
june_2025_results = results_df[results_df['Date'] >= cutoff_date]

# Export results
output_filename = 'Enhanced_VIX_Model_Results_and_Errors_from_June2025.csv'
june_2025_results.to_csv(output_filename, index=False)

print(f"Enhanced results exported to: {output_filename}")
print(f"Results from {june_2025_results['Date'].min().strftime('%Y-%m-%d')} to {june_2025_results['Date'].max().strftime('%Y-%m-%d')}")
print(f"Total predictions: {len(june_2025_results)}")

# Summary statistics for June 2025 onwards
print("\n=== JUNE 2025 ONWARDS PERFORMANCE SUMMARY ===")
if len(june_2025_results) > 0:
    june_metrics = {}
    for model in ['GARCH', 'CNN_LSTM', 'GRU', 'Ensemble']:
        actual = june_2025_results['Actual_VIX'].values
        predicted = june_2025_results[f'{model}_Prediction'].values
        june_metrics[model] = calculate_metrics(actual, predicted)
    
    print(f"{'Model':<15} {'MSE':<10} {'MAE':<10} {'RMSE':<10} {'R²':<10} {'Dir_Acc':<10}")
    print("-" * 75)
    for model_name, metrics in june_metrics.items():
        print(f"{model_name:<15} {metrics['MSE']:<10.4f} {metrics['MAE']:<10.4f} {metrics['RMSE']:<10.4f} {metrics['R2']:<10.4f} {metrics['Directional_Accuracy']:<10.4f}")
else:
    print("No data available for June 2025 onwards period.")

print("\n=== ENHANCED VIX FORECASTING ANALYSIS COMPLETED ===")